In [2]:
import pandas as pd
import numpy as np
import folium
import requests
import pandas as pd
import numpy as np
import pyproj
import polyline
import pickle
import ast
import configparser
from folium import IFrame

/Users/rahimrasool/Documents/Data Science Institute/Perpetual/Summer-23/git-repo/Perpetual/perpetual_env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
config = configparser.ConfigParser()
config.read('config.ini')
mapbox_token = config['mapbox']['token']

In [4]:
filename = 'data/generated_route_list/CAProute_list_20230913_145319.pkl'

with open(filename, 'rb') as file:
    route_data = pickle.load(file)

In [5]:
# convert route_data to a dictionary
route_dict = {}
for i in range(len(route_data)):
    route_dict['Route {}'.format(i+1)] = route_data[i]


In [6]:
# For each key in route_dict, if the list size is greater than 25, split the list into lists of 25
for key in route_dict.keys():
    if len(route_dict[key]) > 25:
        route_dict[key] = [route_dict[key][i:i+25] for i in range(0, len(route_dict[key]), 25)]


In [7]:
# read excel file
df = pd.read_csv('data/Galveston_data/FUE_capacity_high_adoption_with_outdoors.csv')
df.head()

,Name,Weekly_Dropoff_Totes,Daily_Pickup_Totes,Address,Latitude,Longitude
0,Moody Gardens,0.0,0.0,"1 Hope Blvd, Galveston",29.2736,-94.8523
1,Sugar Bean,1.0,1.0,"11 Evia Main, Galveston",29.2610,-94.8704
2,MAC-IES,1.0,1.0,"1110 23rd Street, Galveston",29.2989,-94.7914
3,Good Dough,1.0,1.0,"1508 39th Street, Galveston",29.2906,-94.8085
4,Porch Cafe,1.0,1.0,"1625 East Beach Drive #2A, Galveston",29.3196,-94.7502


In [8]:
# create a dictionary from the df dataframe where the key is the index and value is the name
name_dict = dict(zip(df.index, df['Name']))

# create a dictionary from the df dataframe where the key is the index and value is the pickup_capacity
pickup_capacity_dict = dict(zip(df.index, df['Daily_Pickup_Totes']))

In [9]:
# create a dictionary from the df dataframe where the key is the index and 
# value is combination of latitude and longitude columns combined as a list
lat_long_dict = dict(zip(df.index, df[['Latitude', 'Longitude']].values.tolist()))

### Mapping

In [14]:
# Create a folium map centered at the mean of the locations
m = folium.Map(location=[29.3013, -94.7977], zoom_start=12)

In [15]:
starting_url = "https://api.mapbox.com/directions/v5/mapbox/driving/"
params = {'access_token': mapbox_token, 'overview': 'full'}

total_duration = 0    
for route in route_dict['Route 2']:
    coordinate_lst = []
    for i in route:
        stop_index = list(i.keys())[0]
        accumulated_load = list(i.values())[0]
        coordinates = (lat_long_dict[stop_index][1], lat_long_dict[stop_index][0])
        coordinate_lst.append(coordinates)
    end_url = ";".join([f"{lon},{lat}" for lon, lat in coordinate_lst])
    url = starting_url + end_url
    response = requests.get(url, params=params)
    res = response.json()
    total_duration += res['routes'][0]['duration']

    polyline_str = res['routes'][0]['geometry']
    coords = polyline.decode(polyline_str)

    for i in route:
        stop_index = list(i.keys())[0]
        coordinates = [lat_long_dict[stop_index][0], lat_long_dict[stop_index][1]]
        name = name_dict[stop_index]
        pickup_capacity = pickup_capacity_dict[stop_index]
        html_content = f"<p>Name: {name}</p><p>pickup: {pickup_capacity}</p>"
        iframe = IFrame(html_content, width=200, height=100)
        popup = folium.Popup(iframe, max_width=2650)


        folium.Marker(location= coordinates, icon=folium.Icon(color= 'red', 
                                                                    prefix="fa"),
                                                                    popup=popup
                    ).add_to(m)

    route = folium.PolyLine(locations=coords, color='red', weight=2)
    route.add_to(m)

In [16]:
# save the map as an html file
m.save('data/generated_route_list/CAP_route_now2.html')